<a href="https://colab.research.google.com/github/dguo4/algo_trading/blob/master/starter_files/001_equal_weight_S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests # API call 
import math
import time

In [2]:
!pip install xlsxwriter

import xlsxwriter

     |████████████████████████████████| 153kB 8.8MB/s 


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
# read ticker 
stocks = pd.read_csv('sp_500_stocks.csv')
# stocks

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
# the import function doesn't work on my pc
# from secrets import IEX_CLOUD_API_TOKEN 
IEX_CLOUD_API_TOKEN = 'Tsk_c9ff8a39b60f46c8bd4e37d442d987ed'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' #f string
data = requests.get(api_url).json()
# print(data.status_code)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'A AL/AQSANLL(KSNTDEOEBCM)R TGEGS ', 'calculationPrice': 'close', 'open': 132.58, 'openTime': 1675303517613, 'openSource': 'loifacif', 'close': 136.1, 'closeTime': 1690277744360, 'closeSource': 'flciifoa', 'high': 131.76, 'highTime': 1619774550526, 'highSource': 'u 5e rmynta iddielecpe1', 'low': 132.73, 'lowTime': 1657980041560, 'lowSource': ' cmdt5nr iil1pduea eyee', 'latestPrice': 133.76, 'latestSource': 'Close', 'latestTime': 'January 7, 2021', 'latestUpdate': 1674063063813, 'latestVolume': 114921256, 'iexRealtimePrice': 131.04, 'iexRealtimeSize': 20, 'iexLastUpdated': 1664527735973, 'delayedPrice': 136.81, 'delayedPriceTime': 1639996521322, 'oddLotDelayedPrice': 131.9, 'oddLotDelayedPriceTime': 1643663848216, 'extendedPrice': 136.82, 'extendedChange': 0.77, 'extendedChangePercent': 0.00603, 'extendedPriceTime': 1625439202229, 'previousClose': 130.8, 'previousVolume': 161961268, 'change': 4.44, 'changePercent': 0.0344

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
market_cap/1e12

2.286197384613

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame( columns = my_columns, )

In [9]:
final_dataframe.append(
    pd.Series(
        [symbol, price, market_cap/1e12, 'N/A'], 
        index=my_columns
    ), 
    ignore_index=True, 
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,133.76,2.286197,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [13]:
%%time

final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:10]:
  time.sleep(2)
  print(stock)
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}' #very slow to request using url
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
      pd.Series([stock, data['latestPrice'], data['marketCap']/1e9, 'N/A'],index=my_columns), 
      ignore_index=True
  )

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
CPU times: user 182 ms, sys: 9.89 ms, total: 192 ms
Wall time: 22.2 s


In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.97,39.885459,N/A
1,AAL,15.79,9.753093,N/A
2,AAP,171.81,11.776050,N/A
3,AAPL,130.98,2233.116359,N/A
4,ABBV,108.03,193.696422,N/A
5,ABC,112.61,23.080467,N/A
6,ABMD,335.91,15.084606,N/A
7,ABT,114.00,201.294326,N/A
8,ACN,266.80,180.949409,N/A
9,ADBE,482.13,230.268901,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [15]:
def chunks(lst, n):
  for i in range(0, len(lst), n):
    yield lst[i:i+n]

In [36]:
%%time

symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
# print(symbol_groups)
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))
  # print(symbol_groups[i])
  # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)
# final_dataframe
for symbol_string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
  # print(batch_api_call_url)
  data = requests.get(batch_api_call_url).json()
  # print(data.status_code)
  for symbol in symbol_string.split(','):
    # print(symbol)
    final_dataframe = final_dataframe.append(
        pd.Series([symbol, data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'], index=my_columns),
        ignore_index=True
    )


CPU times: user 1.91 s, sys: 5.38 ms, total: 1.91 s
Wall time: 3.72 s


In [37]:
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.40,39753343130,N/A
1,AAL,16.05,9585030656,N/A
2,AAP,171.95,11769498590,N/A
3,AAPL,132.31,2293597037082,N/A
4,ABBV,109.42,193744985349,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [50]:
portfolio_size = input('Enter the value of your portfolio:')

try:
  val = float(portfolio_size)
except ValueError:
  print('Portfolio need to be a number')
  portfolio_size = input('Enter the value of your portfolio:')
  val = float(portfolio_size) 
  # But it could only give two chances to user to input the correct the number. Or we assume users
  # should be able to provide the right input with our hint. 

Enter the value of your portfolio:1000000


In [56]:
position_size = val/len(final_dataframe.index)
# position_size
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.40,39753343130,14
1,AAL,16.05,9585030656,123
2,AAP,171.95,11769498590,11
3,AAPL,132.31,2293597037082,14
4,ABBV,109.42,193744985349,18
...,...,...,...,...
500,YUM,109.70,32152870126,18
501,ZBH,161.10,33392724646,12
502,ZBRA,425.20,22094927565,4
503,ZION,52.24,8496996125,37


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [62]:
writer = pd.ExcelWriter('recommended trade.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [63]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'font_color': '$0.00',
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'font_color': '0',
        'bg_color': background_color,
        'border': 1
    }
)



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [65]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [66]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
  writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column])
writer.save()

/usr/local/lib/python3.6/dist-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## Saving Our Excel Output

Saving our Excel file is very easy: